<a href="https://colab.research.google.com/github/DiGyt/neuropynamics/blob/lucas_dev/Single_neurons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparations

## Installation of required packages

In [1]:
# We use the brian2 toolbox for our models in the backend
!pip install brian2 -q

     |████████████████████████████████| 1.6MB 7.1MB/s 
     |████████████████████████████████| 5.8MB 19.9MB/s 


## Cloning of GitHub repo

This enables us to access all the models and useful code we created for our toolbox in a Colab notebook

In [2]:
!git clone https://github.com/DiGyt/neuropynamics/ --branch lucas_dev -q


## Imports

In [4]:
# Main packages
import numpy as np

# Brian2 package
# Unit definitions
from brian2 import mV, ms, volt, second, umetre, ufarad, siemens, cm, msiemens, amp, uA, nA
# Other stuff
from brian2 import start_scope, NeuronGroup, StateMonitor, SpikeMonitor, run

# Plotting stuff
import matplotlib.pyplot as plt
import seaborn as sns
from neuropynamics.src.utils.plotting import create_default_plot

# Interactive widgets
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, HBox, VBox, Layout

<IPython.core.display.Javascript object>

## Configuration

In [6]:
# Allow for larger output cells
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'''))

# Set button layout
button_layout = Layout(width='180px', height='30px')

<IPython.core.display.Javascript object>

# Simulations of single neurons

## Perfect Integrator

To give you a short introduction to the brian2 toolbox (https://briansimulator.org) that we will be using for the simulation of our neurons, lets take a look at the perfect integrator neuron and how it can be implemented using brian2.

Do not worry if you don't get everything right away, we designed our toolbox in a way that allows you to play around with the simulations without having to write or change a single line of code.

In brian2, every neuron is defined by a set of differential equations. In addition, the threshold for spiking is defined as an equation as well as the reset behavior after spiking.

For the perfect integrator, the equation is quite simple: $dV_m/dt = \frac{I}{\tau}$ where $V_m$ is the membrane potential, $I$ is the external current and the membrane time constant $\tau$ modulates the charging of the integrator. 

The neuron spikes if the membrane potential $V_m$ exceeds the threshold $V_{max}$ and then the membrane potential is reset to a value below the threshold (in our case 20mV below that).

Let's take a look at how this looks in our code:

```
def create_perfect_integrator_neuron(Vmax):
    """Creates a brian2 NeuronGroup that contains a single perfect integrator neuron"""
    # Define differential equation for leaky integrate-and-fire neuron
    eqs = '''   
        dvm/dt = I/tau : volt
        I : volt
        '''
    # Define reset function
    reset = 'vm = {}*mV'.format(Vmax-20)
    # Define threshold
    threshold = 'vm > {}*mV'.format(Vmax)
    # Return NeuronGroup object
    return NeuronGroup(1, eqs, threshold = threshold, reset = reset, method = 'euler')
```



As you can see, SI units imported from the brian2 package (see above) are "multiplied" with a variable to denote its unit in these equations. This is the way brian2 integrates units within Python and you will see it a lot throughout this code. The equations also specify the resulting unit for each equation. 

Also, as brian2 is mostly used for modeling of larger-scale models instead of single neurons, we have to create a "NeuronGroup" of one neuron defined by the equations, threshhold, reset behavior, and the method of numerical integration used to solve the equations.

After the creation of a neuron (or technically a group of a single neuron), we can then define the neurons parameters dynamically in the code. For the perfect integrator, $\tau$ regulates the charge of neuron over time. The last line of the code snippet from the cell below sets the value of $\tau$ to the function parameter.

```
def create_and_plot_perfect_integrator_neuron(I_ext, tau, Vmax):
  # Start the scope for the brian2 toolbox to register all neurons that are created
  start_scope()
  # Define the neuron
  neuron = create_perfect_integrator_neuron(Vmax)      
  # Set neuron parameters
  tau = tau*ms;
```

Again, we need to define the unit of our $\tau$ by multiplying it with the unit "ms" imported from brian2. The rest of the code in the function is used to set up monitoring of the neurons state and spiking and injecting a current. Everything outside the function is used for creating an interactive plot to play around with the model.

That's it for the short intro to brian2. You can see the perfect integrator in action by executing the cell below. You may change the external current and tau using the slides and see for yourselves how that influences the neurons behavior. Please be aware that especially for more complex models, the plot might take a bit to update.

For more information about brian2, see the [official tutorials](https://brian2.readthedocs.io/en/stable/resources/tutorials/1-intro-to-brian-neurons.html).

In [7]:
# Import the wrapper function to create a leaky integrate-and-fire neuron easily in brian2 
from neuropynamics.src.models.neurons import create_perfect_integrator_neuron

# Create function that creates a neuron and plots its behavior based on the given parameters
def create_and_plot_perfect_integrator_neuron(I_ext, tau, Vmax):
  # Start the scope for the brian2 toolbox to register all neurons that are created
  start_scope()
  # Define the neuron
  neuron = create_perfect_integrator_neuron(Vmax)      
  # Set neuron parameters
  tau = tau*ms;
  # Start monitoring the neurons state
  statemon = StateMonitor(source = neuron, variables = ['vm', 'I'], record = 0)
  # Start monitoring spiking behavior
  spikemon = SpikeMonitor(source = neuron, variables= 'vm')
  # Run neuron simulation for 100ms without input
  run(100*ms)
  # Set input current to neuron
  neuron.I = I_ext * mV
  # Run 500ms with input
  run(500*ms)
  # Remove input current to neuron
  neuron.I = 0 * mV
  # Run neuron simulation for 100ms without input
  run(100*ms)
  # Plot results
  create_default_plot(x = statemon.t/ms, neuron_data = [statemon.vm[0]/mV], neuron_labels = ['Membrane Potential (mV)'], neuron_colors = ['steelblue'], spikes = [spikemon.t/ms, spikemon.vm/mV], spike_color = 'steelblue', input_current = statemon.I[0]/mV, input_label = 'Input Current (mV)', input_color = 'gold', y_range = [-80,-40], title = 'Perfect Integrator Neuron', x_axis_label = 'Time (ms)', y_axis_label = 'Membrane Potential (mV)', input_axis_label = 'Input Current (mV)', hline = Vmax)
  
# Set default parameters 
I_ext_def = 10.
tau_def = 20.

# Create sliders for neuron parameters
tau_slider = widgets.FloatSlider(value = tau_def, min = 1., max = 30., step = 1, description = 'tau:', readout_format = '.1f', continuous_update = False)

# Create slider for input current
I_slider = widgets.FloatSlider(value = I_ext_def, min = 0., max = 40., step = 1, description = 'I:', readout_format = '.1f', continuous_update = False)

# Make interactive widget for function above with the given sliders
main_widgets = interactive(create_and_plot_perfect_integrator_neuron, I_ext = I_slider, tau = tau_slider, Vmax = fixed(-50))

def reset_perfect_integrator(name):
  I_ext_slider.value = I_ext_def
  tau_slider.value = tau_def

# Reset button
reset_button = widgets.Button(description='Reset', layout = button_layout)
reset_button.on_click(reset_perfect_integrator)

# Display main widgets and reset button
display(VBox(children=[reset_button, main_widgets]))

## Leaky Integrate-and-Fire Model

Now after having examined our perfect integrator, let's drag it down to reality, where nothing really is perfect (and that is the beauty of it). 

Neurons have channels on their membrane that allow ions to pass through it and therefore they are leaky. In order to reflect this, we just have to alter our equations a bit to amount for the leakage:

$dV_m/dt = \frac{E_{leak} - V_m + I}{\tau}$  

Looking at this through the brian2 code lens, this results in the following:


```
def create_lif_neuron(Vmax):
    """Creates a brian2 NeuronGroup that contains a single leaky integrate-and-fire neuron"""
    # Define differential equation for leaky integrate-and-fire neuron
    eqs = '''   
        dvm/dt = ((El - vm) + I)/tau : volt
        I : volt
        '''
    # Define reset function
    reset = 'vm = {}*mV'.format(Vmax-20)
    # Define threshold
    threshold = 'vm > {}*mV'.format(Vmax)
    # Return NeuronGroup object
    return NeuronGroup(1, eqs, threshold = threshold, reset = reset, method = 'euler')
```

The reset and threshold functions are still the same and we included the changed differential equations that amounts for the leakage. Now we just need to set "El" in the neuron parameter section and we are good to go.



```
def create_and_plot_lif_neuron(I_ext, tau, e_leak, Vmax):
  # Start the scope for the brian2 toolbox to register all neurons that are created
  start_scope()
  # Define the neuron
  neuron = create_lif_neuron(Vmax)      
  # Set neuron parameters
  tau = tau*ms;
  El = -e_leak*mV
```

Execute the following cell and see how different leakage affects the behavior of our model.

In [15]:
# Import the wrapper function to create a leaky integrate-and-fire neuron easily in brian2 
from neuropynamics.src.models.neurons import create_lif_neuron

# Create function that creates a neuron and plots its behavior based on the given parameters
def create_and_plot_lif_neuron(I_ext, tau, e_leak, Vmax):
  # Start the scope for the brian2 toolbox to register all neurons that are created
  start_scope()
  # Define the neuron
  neuron = create_lif_neuron(Vmax)      
  # Set neuron parameters
  tau = tau*ms;
  El = -e_leak*mV
  # Start monitoring the neurons state
  statemon = StateMonitor(source = neuron, variables = ['vm', 'I'], record = 0)
  # Start monitoring spiking behavior
  spikemon = SpikeMonitor(source = neuron, variables= 'vm')
  # Run neuron simulation for 100ms without input
  run(100*ms)
  # Set input current to neuron
  neuron.I = I_ext * mV
  # Run 500ms with input
  run(500*ms)
  # Remove input current to neuron
  neuron.I = 0 * mV
  # Run neuron simulation for 100ms without input
  run(100*ms)
  # Plot results
  create_default_plot(x = statemon.t/ms, neuron_data = [statemon.vm[0]/mV], neuron_labels = ['Membrane Potential (mV)'], neuron_colors = ['steelblue'], spikes = [spikemon.t/ms, spikemon.vm/mV], spike_color = 'steelblue', input_current = statemon.I[0]/mV, input_label = 'Input Current (mV)', input_color = 'gold', y_range = [-90,0], title = 'Leaky Integrate-and-Fire Neuron', x_axis_label = 'Time (ms)', y_axis_label = 'Membrane Potential', input_axis_label = 'Input Current (mV)', hline = Vmax)
  
# Set default parameters 
I_ext_def = 10.
tau_def = 20.
e_leak_def = 59.

# Create sliders for neuron parameters
tau_slider = widgets.FloatSlider(value = tau_def, min = 1., max = 30., step = 1, description = 'tau:', readout_format = '.1f', continuous_update = False)
e_leak_slider = widgets.FloatSlider(value = e_leak_def, min = 35., max = 85., step = 1, description = 'E_leak:', readout_format = '.1f', continuous_update = False) 

# Create slider for input current
I_ext_slider = widgets.FloatSlider(value = I_ext_def, min = 0., max = 40., step = 1, description = 'I:', readout_format = '.1f', continuous_update = False)

# Make interactive widget for function above with the given sliders
main_widgets = interactive(create_and_plot_lif_neuron, I_ext = I_ext_slider, tau = tau_slider, e_leak = e_leak_slider, Vmax = fixed(-50))

def reset_lif(name):
  I_ext_slider.value = I_ext_def
  tau_slider.value = tau_def
  e_leak_slider.value = e_leak_def

# Reset button
reset_button = widgets.Button(description='Reset', layout = button_layout)
reset_button.on_click(reset_lif)

# Display main widgets and reset button
display(VBox(children=[reset_button, main_widgets]))

## Izhikevich model

Now let's take a look at a model that is a bit more complex as it uses a set of equations rather than one line for the behavior of a neuron: The Izhikevich model. 

As you may recall from the lecture, the Izhikevich model uses the ... TODO

In [51]:
# Import the wrapper function to create a Izhikevich neuron easily in brian2 
from neuropynamics.src.models.neurons import create_izhikevich_neuron

# Create function that creates a neuron and plots its behavior based on the given parameters
def create_and_plot_izhikevich_neuron(I_ext, a, b, c, d, Vmax):
  # Start the scope for the brian2 toolbox to register all neurons that are created
  start_scope()
  # Define the neuron
  neuron = create_izhikevich_neuron(Vmax)      
  # Set neuron parameters
  a = a/ms
  b = b/ms
  c = c * mV 
  d = d * volt/second  
  # Start monitoring the neurons state
  statemon = StateMonitor(source = neuron, variables = ['vm', 'w', 'I'], record = 0)
  # Start monitoring spiking behavior
  spikemon = SpikeMonitor(source = neuron, variables= 'vm')
  # Run neuron simulation for 100ms without input
  run(100*ms)
  # Set input current to neuron
  neuron.I = I_ext * volt / second
  # Run 500ms with input
  run(500*ms)
  # Remove input current to neuron
  neuron.I = 0 * volt / second
  # Run neuron simulation for 100ms without input
  run(100*ms)
  # Plot results
  create_default_plot(x = statemon.t/ms, neuron_data = [statemon.vm[0]/mV, statemon.w[0]], neuron_labels = ['Membrane Potential (mV)', 'Voltage change (V / s)'], neuron_colors = ['steelblue', 'mediumseagreen'], spikes = [spikemon.t/ms, spikemon.vm/mV], spike_color = 'steelblue', input_current = statemon.I[0], input_label = 'Input Current (A)', input_color = 'gold', y_range = [-100,40], title = 'Izhikevich Neuron', x_axis_label = 'Time (ms)', y_axis_label = 'Membrane Potential / Voltage Change', input_axis_label = 'Input Current', hline = Vmax)
  
# Set default parameters 
I_ext_def = 10.
a_def = 0.02
b_def = 0.2
c_def = -65.
d_def = 8.
vmax_def = -5.

# Create sliders for neuron parameters
a_slider = widgets.FloatSlider(value = a_def, min = 0., max = 0.15, step = 0.01, description = 'a:', readout_format = '.2f', continuous_update = False)
b_slider = widgets.FloatSlider(value = b_def, min = 0., max = 0.35, step = 0.01, description = 'b:', readout_format = '.2f', continuous_update = False) # Somehow max = 0.3 does not work
c_slider = widgets.FloatSlider(value = c_def, min = -75., max = -40., step = 1, description = 'c:', readout_format = '.1f', continuous_update = False)
d_slider = widgets.FloatSlider(value = d_def, min = 0., max = 10., step = 1, description = 'd:', readout_format = '.1f', continuous_update = False)
vmax_slider = widgets.FloatSlider(value = vmax_def, min = -35., max = 15., step = 1, description = 'Vmax:', readout_format = '.1f', continuous_update = False)

# Create slider for input current
I_slider = widgets.FloatSlider(value = I_ext_def, min = 0., max = 40., step = 1, description = 'I:', readout_format = '.1f', continuous_update = False)

# Make interactive widget for function above with the given sliders
main_widgets = interactive(create_and_plot_izhikevich_neuron, I_ext = I_slider, a = a_slider, b = b_slider, c = c_slider, d = d_slider, Vmax = vmax_slider)

# Create functions to set specific neuron configurations
def apply_config(I_ext, a, b, c, d, vmax):
  I_slider.value = I_ext
  a_slider.value = a
  b_slider.value = b
  c_slider.value = c
  d_slider.value = d
  vmax_slider.value = vmax

def reset_config(name):
  apply_config(I_ext = I_ext_def, a = a_def, b = b_def, c = c_def, d = d_def, vmax = vmax_def)

def apply_intrinsically_bursting_config(name):
  apply_config(I_ext = 12., a = a_def, b = b_def, c = -55, d = 4, vmax = vmax_def)

def apply_chattering_config(name):
  apply_config(I_ext = I_ext_def, a = a_def, b = b_def, c = -50, d = 2, vmax = vmax_def)

def apply_fast_spiking_config(name):
  apply_config(I_ext = I_ext_def, a = 0.1, b = b_def, c = c_def, d = d_def, vmax = vmax_def)

def apply_low_thresh_spiking_config(name):
  apply_config(I_ext = I_ext_def, a = a_def, b = 0.25, c = c_def, d = d_def, vmax = vmax_def)

def apply_resonator_config(name):
  apply_config(I_ext = I_ext_def, a = 0.1, b = 0.26, c = c_def, d = d_def, vmax = vmax_def)

# Create buttons for specific neuron configurations
# Reset button
regular_spiking_button = widgets.Button(description='Regular Spiking', layout = button_layout)
regular_spiking_button.on_click(reset_config)
# Intrinsically bursting
intrinsically_bursting_button = widgets.Button(description='Intrinsically Bursting', layout = button_layout)
intrinsically_bursting_button.on_click(apply_intrinsically_bursting_config)
# Chattering neuron
chattering_button = widgets.Button(description='Chattering', layout = button_layout)
chattering_button.on_click(apply_chattering_config)
# Fast spiking
fast_spiking_button = widgets.Button(description='Fast Spiking', layout = button_layout)
fast_spiking_button.on_click(apply_fast_spiking_config)
# Low-threshold spiking
low_thresh_spiking_button = widgets.Button(description='Low-threshold Spiking', layout = button_layout)
low_thresh_spiking_button.on_click(apply_low_thresh_spiking_config)
# Resonator
resonator_button = widgets.Button(description='Resonator', layout = button_layout)
resonator_button.on_click(apply_resonator_config)

# Place buttons into grid
button_description = widgets.Label(value='Select predefined neuron types:', layout = Layout(width='300px', height='30px'))
buttons = HBox(children=[regular_spiking_button, intrinsically_bursting_button, chattering_button, fast_spiking_button, low_thresh_spiking_button, resonator_button])

# Display main widget and buttons
parameter_description = widgets.Label(value='Set parameters yourself:', layout = Layout(width='300px', height='30px'))
display(VBox(children=[VBox(children=[button_description, buttons]), VBox(children=[parameter_description, main_widgets])]))

## Hodgkin-Huxley model

In [52]:
# Import the wrapper function to create a Hodgkin-Huxley neuron easily in brian2 
from neuropynamics.src.models.neurons import create_hodgkin_huxley_neuron

# Create function that creates a neuron and plots its behavior based on the given parameters
def create_and_plot_hh_neuron(I_ext, g_leak, g_k, g_na, e_leak, e_k, e_na, Vmax):
  # Start the scope for the brian2 toolbox to register all neurons that are created
  start_scope()
  # Define the neuron
  neuron = create_hodgkin_huxley_neuron(Vmax)      
  # Set neuron parameters
  area = 20000*umetre**2
  Cm = 1*ufarad*cm**-2 * area # Mebrane capacitance
  gl = g_leak*siemens*cm**-2 * area
  El = e_leak*mV
  EK = e_k*mV
  ENa = e_na*mV
  g_na = g_na*msiemens*cm**-2 * area
  g_kd = g_k*msiemens*cm**-2 * area
  VT = -63*mV
  # Set initial voltage to leak conductance
  neuron.v = El  
  # Start monitoring the neurons state
  statemon = StateMonitor(source = neuron, variables = ['v', 'I', 'm', 'n', 'h'], record = 0)
  # Start monitoring spiking behavior
  spikemon = SpikeMonitor(source = neuron, variables= 'v')
  # Run neuron simulation for 10ms without input
  run(10*ms)
  # Set input current to neuron
  neuron.I = I_ext * nA
  # Run 100ms with input
  run(100*ms)
  # Remove input current to neuron
  neuron.I = 0 * nA
  # Run neuron simulation for 10ms without input
  run(10*ms)
  # Plot results  
  create_default_plot(x = statemon.t/ms, neuron_data = [statemon.v[0]/mV], neuron_labels = ['Membrane Potential (mV)'], neuron_colors = ['steelblue'], spikes = [spikemon.t/ms, spikemon.v/mV], spike_color = 'steelblue', input_current = statemon.I[0], input_label = 'Input Current (nA)', input_color = 'gold', y_range = None, title = 'Hodgkin-Huxley Neuron', x_axis_label = 'Time (ms)', y_axis_label = 'Membrane Potential', input_axis_label = 'Input Current', hline = Vmax)
  # Plot gating values
  create_default_plot(x = statemon.t/ms, neuron_data = [statemon.m[0], statemon.n[0], statemon.h[0]], neuron_labels = ['m', 'n', 'h'], neuron_colors = ['red', 'green', 'blue'], x_axis_label = 'Time (ms)', y_axis_label = 'Gating Values')

# Set default parameters 
I_ext_def = 0.3
g_leak_def = 5e-5
g_k_def = 30.
g_na_def = 100.
e_leak_def = -65.
e_k_def = -90.
e_na_def = 50.
vmax_def = -40.

# Create sliders for neuron parameters
g_leak_slider = widgets.FloatSlider(value = g_leak_def, min = 0., max = 10e-5, step = 1e-6, description = 'g_leak:', readout_format = '.5f', continuous_update = False)
g_k_slider = widgets.FloatSlider(value = g_k_def, min = 0., max = 70., step = 1, description = 'g_k:', readout_format = '.1f', continuous_update = False)
g_na_slider = widgets.FloatSlider(value = g_na_def, min = 80., max = 160., step = 1, description = 'g_na:', readout_format = '.1f', continuous_update = False)
e_leak_slider = widgets.FloatSlider(value = e_leak_def, min = -70., max = -40., step = 1, description = 'E_leak:', readout_format = '.1f', continuous_update = False)
e_k_slider = widgets.FloatSlider(value = e_k_def, min = -100., max = -50., step = 1, description = 'E_k:', readout_format = '.1f', continuous_update = False)
e_na_slider = widgets.FloatSlider(value = e_na_def, min = 20., max = 80., step = 1, description = 'E_na:', readout_format = '.1f', continuous_update = False)
vmax_slider = widgets.FloatSlider(value = vmax_def, min = -55., max = 15., step = 1, description = 'Vmax:', readout_format = '.1f', continuous_update = False)

# Create slider for input current
I_ext_slider = widgets.FloatSlider(value = I_ext_def, min = 0., max = 1., step = 0.01, description = 'I:', readout_format = '.2f', continuous_update = False)

# Make interactive widget for function above with the given sliders
main_widgets = interactive(create_and_plot_hh_neuron, I_ext = I_ext_slider, g_leak = g_leak_slider, g_k = g_k_slider, g_na = g_na_slider, e_leak = e_leak_slider, e_k = e_k_slider, e_na = e_na_slider, Vmax = vmax_slider)

# Create function to reset values
def reset_hh(name):
  I_ext_slider.value = I_ext_def
  g_k_slider.value = g_k_def
  g_leak_slider.value = g_leak_def
  g_na_slider.value = g_na_def
  e_leak_slider.value = e_leak_def
  e_k_slider.value = e_k_def
  e_na_slider.value = e_na_def

# Reset button
reset_button = widgets.Button(description='Reset', layout = button_layout)
reset_button.on_click(reset_hh)

# Display main widgets and reset button
display(VBox(children=[reset_button, main_widgets]))